# Async programming

## I/O 대기
+ CPU 클럭의 속도가 1초라고 가정했을 때
+ RAM은 20초, SSD는 4일, HDD 6개월, 네트워크 전송은?

## 동시성
+ 여러 개의 요청을 동시에 다룰 수 있는 시스템을 구현하는 방식
+ 아래는 동기식으로 구현된 기본 함수

In [1]:
import time 
import functools


DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}, {kwargs}) -> {result}'

def clock(fmt=DEFAULT_FMT): 
    def decorate(func): 
        @functools.wraps(func)
        def clocked(*_args, **_kwargs): # clocked에서 *, ** 키워드를 통해 설정된 인수를 변수화
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            pairs = ['%s=%r' % (k, w) for k, w in sorted(_kwargs.items())]
            kwargs = ', '.join(pairs)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result # clocked()는 데커레이트된 함수를 대체하므로, 원래 함수가 반환하는 값을 반환해야 한다.
        return clocked     # decorate()는 clocked()를 반환한다. 
    return decorate        # clock()은 decorate()를 반환한다. 


def network_request(number):
    time.sleep(1.0)
    return {"success": True, "result": number ** 2}


@clock()
def fetch_square(number):
    response = network_request(number)
    if response["success"]:
        return response["result"]

        
if __name__ == "__main__":
    t0 = time.time()

    returns = [fetch_square(i) for i in range(10)]
    
    elapsed = time.time() - t0
    print("Total:", elapsed, "sec")

[1.00106335s] fetch_square(0, ) -> 0
[1.00108242s] fetch_square(1, ) -> 1
[1.00107408s] fetch_square(2, ) -> 4
[1.00107789s] fetch_square(3, ) -> 9
[1.00108576s] fetch_square(4, ) -> 16
[1.00108337s] fetch_square(5, ) -> 25
[1.00105596s] fetch_square(6, ) -> 36
[1.00109434s] fetch_square(7, ) -> 49
[1.00107479s] fetch_square(8, ) -> 64
[1.00108171s] fetch_square(9, ) -> 81
Total: 10.012474536895752 sec


### Callback

+ 원리만 이해하고 복잡하니 쓰지 말자

In [2]:
import threading


def network_request_async(number, on_done):
    
    def timer_done():
        time.sleep(1.0)
        on_done({"success": True, "result": number ** 2})

    timer = threading.Thread(target=timer_done, args=[])
    timer.start()
    
    
@clock()
def fetch_square_async(number):
    
    def on_done(response):
        if response["success"]:
            return response["result"]
        
    network_request_async(number, on_done)

In [3]:
if __name__ == "__main__":
    t0 = time.time()

    returns = [fetch_square_async(i) for i in range(10)]
    
    elapsed = time.time() - t0
    print("Total:", elapsed, "sec")

[0.00020146s] fetch_square_async(0, ) -> None
[0.00047803s] fetch_square_async(1, ) -> None
[0.00028396s] fetch_square_async(2, ) -> None
[0.00019121s] fetch_square_async(3, ) -> None
[0.00026870s] fetch_square_async(4, ) -> None
[0.00024271s] fetch_square_async(5, ) -> None
[0.00013614s] fetch_square_async(6, ) -> None
[0.00022125s] fetch_square_async(7, ) -> None
[0.00043344s] fetch_square_async(8, ) -> None
[0.00015450s] fetch_square_async(9, ) -> None
Total: 0.003396749496459961 sec


### Future

+ 요청한 자원을 추적하고 가용하게 될 때 까지 대기하는 데 도움이 되는 추상화

In [21]:
from concurrent.futures import Future

def callback(future):
    print(future.result()[::-1])

fut = Future()
fut.add_done_callback(callback) 

In [22]:
vars(fut)

{'_condition': <Condition(<unlocked _thread.RLock object owner=0 count=0 at 0x7fc5b5fdab10>, 0)>,
 '_state': 'PENDING',
 '_result': None,
 '_exception': None,
 '_waiters': [],
 '_done_callbacks': [<function __main__.callback(future)>]}

In [23]:
fut.set_result("HELLO") ## 퓨쳐를 리턴함

OLLEH


In [24]:
vars(fut)

{'_condition': <Condition(<unlocked _thread.RLock object owner=0 count=0 at 0x7fc5b5fdab10>, 0)>,
 '_state': 'FINISHED',
 '_result': 'HELLO',
 '_exception': None,
 '_waiters': [],
 '_done_callbacks': [<function __main__.callback(future)>]}

+ network_request_async

In [52]:
def network_reqeust_async(number):
    future = Future()
    result = {"success": True, "result": number ** 2}
    timer = threading.Timer(1.0, lambda: future.set_result(result))
    timer.start()
    return future


@clock()
def fetch_square_async(number):
    fut = network_reqeust_async(number)

    def on_done_future(future):
        response = future.result()
        if response["success"]:
#             return response["result"]
            print(response["result"], flush=True)        
        
    fut.add_done_callback(on_done_future)

In [53]:
if __name__ == "__main__":
    t0 = time.time()

    returns = [fetch_square_async(i) for i in range(10)]
    
    elapsed = time.time() - t0
    print("Total:", elapsed, "sec")

[0.00039458s] fetch_square_async(0, ) -> None
[0.00022340s] fetch_square_async(1, ) -> None
[0.00046802s] fetch_square_async(2, ) -> None
[0.00025463s] fetch_square_async(3, ) -> None
[0.00025582s] fetch_square_async(4, ) -> None
[0.00025749s] fetch_square_async(5, ) -> None
[0.00018716s] fetch_square_async(6, ) -> None
[0.00017023s] fetch_square_async(7, ) -> None
[0.00016737s] fetch_square_async(8, ) -> None
[0.00017452s] fetch_square_async(9, ) -> None
Total: 0.003279447555541992 sec
0
436
1
25
49
9
64
81

16


## asyncio 프레임워크
+ asyncio.get_event_loop() 함수를 호출해 asyncio 루프를 얻을 수 있음
+